In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, matthews_corrcoef
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from src.preprocessing import make_dataset, recover_split
from src.utils import read_jsonl
import numpy as np
import seaborn as sns

In [ ]:
DATASET_NAME = "CreativeLang/EPIC_Irony"
df = make_dataset(pd.DataFrame(load_dataset(DATASET_NAME)['train']))

In [ ]:
def get_golds_preds(split):
    golds = [item['gold'] for item in split]
    preds = [item['pred'] for item in split]
    return golds, preds

def evaluate_result(result):
    outputs = []
    for split in result:
        outputs.append(evaluate_split(split))
    return outputs

def evaluate_split(split):
    golds, preds = get_golds_preds(split)
    outputs = classification_report(golds, preds, output_dict=True)
    outputs['mcc'] = matthews_corrcoef(golds, preds)
    return outputs

def group_metric(metrics, k, kk=None):
    if kk==None:
        values = [m[k] for m in metrics]
    else:
        values = [m[k][kk] for m in metrics]
    return np.round(np.min(values), 4), np.round(np.mean(values), 4), np.round(np.max(values), 4), np.round(np.std(values), 4)

def compute_metric_stats(metrics):
    for k, v in metrics[0].items():
        if isinstance(v, dict):
            for kk, vv in v.items():
                print(k, kk, group_metric(metrics, k, kk))
        else:
            print(k, group_metric(metrics, k))

In [ ]:
data = read_jsonl('results/clm_llama_nt.jsonl')
metrics = evaluate_result(data)
compute_metric_stats(metrics)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(*get_golds_preds(data[0]))).plot()

In [ ]:
recover_split([item['id_original'] for item in read_jsonl('results/clm_llama_nt.jsonl')[1] if item['gold'] == 1 and item['pred'] == 1], df)

## Losses 

In [ ]:
def plot_loss(states):
    best_split = np.argmax([state['best_metric'] for state in states])
    states[best_split]
    losses = []

    for i in range(0, len(states[best_split]['log_history'])-1, 2):
        losses.append((i / 2, states[best_split]['log_history'][i]['loss'], 'train'))
        losses.append((i / 2, states[best_split]['log_history'][i+1]['eval_loss'], 'val'))
    

    losses_df = pd.DataFrame(losses, columns=['epoch', 'loss', 'train_val'])

    sns.lineplot(
        data = losses_df,
        x = 'epoch',
        y = 'loss',
        hue = 'train_val'
    )
    

In [ ]:
states = read_jsonl('results/roberta-irony-ft_loss.jsonl')
plot_loss(states)